In [2]:
# Dependencies
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [3]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\bluec\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [4]:
# URL of page to be scraped
# url = 'http://house.speakingsame.com/p.php?q=Perth%2C+WA'
url = 'http://house.speakingsame.com/p.php?q=Nedlands%2C+WA'
browser.visit(url)

In [ ]:
addresses = []
properties_info = []

pages = np.arange(0, 30, 1)

for page in pages:
    
#     page_url = "http://house.speakingsame.com/p.php?q=Perth&p="+ str(page) +"&s=1&st=&type=&count=300&region=Perth&lat=0&lng=0&sta=wa&htype=&agent=0&minprice=0&maxprice=0&minbed=0&maxbed=0&minland=0&maxland=0"
    page_url = "http://house.speakingsame.com/p.php?q=nedlands&p="+ str(page) +"&s=1&st=&type=&count=300&region=nedlands&lat=0&lng=0&sta=wa&htype=&agent=0&minprice=0&maxprice=0&minbed=0&maxbed=0&minland=0&maxland=0"
    
    browser.visit(page_url)
   
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    addr_spans = soup.find_all('span', class_='addr')

    for span in addr_spans:
        addresses.append(span.a.text)
    addresses

    tables = soup.find_all('table', style="font-size:13px") #tables list (except 0,1 index) contians list of properties in the page

    for i in range(2,12):
        try:
            info = []
            trs = tables[i].find_all('tr')
            for tr in trs:
                info.append(tr.td.text)
        except:
            info.append('')
        properties_info.append(info)
        
  
    
properties_info  

In [8]:
len(properties_info)

300

In [10]:
len(addresses)

120

In [11]:
Properties = []
for i in range(0, len(addresses)):
    
    Property_dict = {}
    Property_dict['address'] = addresses[i]
    
    for j in range (0, len(properties_info[i])):
        
        try:
                    
            if properties_info[i][j].split(' ')[0] == "Sold": 
                Property_dict['price'] = properties_info[i][j].split(' ')[1]
                Property_dict['sold date'] = properties_info[i][j].split('in ')[1]

            if properties_info[i][j].split(' ')[0] in ["Apartment:", "House:", "Townhouse:"]:
                Property_dict['type'] = properties_info[i][j].split(' ')[0].split(':')[0]
                Property_dict['bedrooms'] = properties_info[i][j].split(' ')[1]
                try:
                    Property_dict['bathrooms'] = properties_info[i][j].split(' ')[3]
                except:
                    Property_dict['bathrooms'] = ''
                try:
                    Property_dict['car space'] = properties_info[i][j].split(' ')[5]
                except:
                    Property_dict['car space'] = ''
                    
            if properties_info[i][j].split(' ')[0] == "Agent:":
                Property_dict['agent'] = properties_info[i][j].split(': ')[1]
                
            if properties_info[i][j].split(' ')[0] == "Rent":
                Property_dict['rent'] = properties_info[i][j].split(' ')[1]
                Property_dict['rent date'] = properties_info[i][j].split('in ')[1]
                
            if properties_info[i][j].split(' ')[1] == "size:":
                if properties_info[i][j].split(' ')[0] == "Land":
                    Property_dict['Land size'] = properties_info[i][j].split(' ')[2]
                    try:
                        if properties_info[i][j].split(' ')[5] == "Building":
                            Property_dict['Building size'] = properties_info[i][j].split(' ')[7]
                    except:
                        continue
                elif properties_info[i][j].split(' ')[0] == "Building":
                    Property_dict['Building size'] = properties_info[i][j].split(' ')[2]
            
            
        except:
            continue

  
    Properties.append(Property_dict)
        
Properties

[{'address': '318/2 Moreau Parade',
  'price': '$539,000',
  'sold date': 'Aug 2021',
  'type': 'Apartment',
  'bedrooms': '',
  'bathrooms': '',
  'car space': '',
  'Land size': '110'},
 {'address': '54/193 Hay Street',
  'price': '$295,000',
  'sold date': 'Aug 2021',
  'type': 'Apartment',
  'bedrooms': '2',
  'bathrooms': '1',
  'car space': '1',
  'Land size': '2,425',
  'Building size': '99',
  'agent': 'Joel Cooper'},
 {'address': '30 Joel Terrace',
  'price': '$1,100,000',
  'sold date': 'Jul 2021',
  'type': 'House',
  'bedrooms': '4',
  'bathrooms': '',
  'car space': '',
  'Land size': '454',
  'agent': 'Nadija Begovich'},
 {'address': '10/38 Fielder Street',
  'price': '$429,000',
  'sold date': 'Jul 2021',
  'rent': '$450pw',
  'rent date': 'Sep 2018',
  'Land size': '131',
  'Building size': '150',
  'agent': 'Brendan Smith'},
 {'address': '36/78 Terrace Road',
  'price': '$700,000',
  'sold date': 'Jul 2021',
  'rent': '$800pw',
  'rent date': 'Jul 2013',
  'type': 'Apa

In [12]:
len(Properties)

120

In [13]:
nedlands_df = pd.DataFrame(Properties)

In [14]:
nedlands_df

,address,price,sold date,type,bedrooms,bathrooms,car space,Land size,Building size,agent,rent,rent date
0,318/2 Moreau Parade,"$539,000",Aug 2021,Apartment,,,,110,NaN,NaN,NaN,NaN
1,54/193 Hay Street,"$295,000",Aug 2021,Apartment,2,1,1,"2,425",99,Joel Cooper,NaN,NaN
2,30 Joel Terrace,"$1,100,000",Jul 2021,House,4,,,454,NaN,Nadija Begovich,NaN,NaN
3,10/38 Fielder Street,"$429,000",Jul 2021,NaN,NaN,NaN,NaN,131,150,Brendan Smith,$450pw,Sep 2018
4,36/78 Terrace Road,"$700,000",Jul 2021,Apartment,2,2,2,"6,691",100,Jasmyn Wright,$800pw,Jul 2013
...,...,...,...,...,...,...,...,...,...,...,...,...
115,6/128 Brown Street,"$418,000",Feb 2021,Apartment,2,2,1,"1,018",112,Clinton Edwards,$440pw,Oct 2016
116,301/108 Bennett Street,"$380,000",Feb 2021,Apartment,2,2,1,NaN,NaN,NaN,$400pw,Jun 2020
117,83/128 Adelaide Terrace,"$418,500",Feb 2021,NaN,NaN,NaN,NaN,111,82,NaN,$430pw,Apr 2018
118,Apt. 29/98 Terrace Road,"$690,000",Feb 2021,Apartment,2,2,2,108,NaN,Peter Wright,NaN,NaN


In [15]:
nedlands_df.to_csv("EastPerth_suburb_properties.csv")